## A gentle 10-minute introduction to Ray AI Runitime (Ray AIR)

© 2019-2022, Anyscale. All Rights Reserved

📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
⬅️ [Previous notebook](./ex_07_ray_data.ipynb) <br>

As part of Ray 2.0, Ray AI Runtime (AIR) is an open-source toolkit for building end-to-end simple and scalable ML applications. By leveraging Ray, its distributed compute capabilities, and its library ecosystem, Ray AIR brings scalability and programmability to ML platforms.

Ray AI Runtime focuses on two functional aspects:
 * It provides scalability by leveraging Ray’s distributed compute layer for ML workloads.
 * It is designed to interoperate with other systems for storage and metadata needs.

Ray AIR consists of five key components:

 * Data processing ([Ray Data](https://docs.ray.io/en/latest/data/dataset.html))
 * Model Training ([Ray Train](https://docs.ray.io/en/latest/train/train.html))
 * Reinforcement Learning ([Ray RLlib](https://docs.ray.io/en/latest/rllib/index.html))
 * Hyperparameter Tuning ([Ray Tune](https://docs.ray.io/en/latest/tune/index.html))
 * Model Serving ([Ray Serve](https://docs.ray.io/en/latest/serve/index.html)).
 
 <img src = "images/ray-air.svg" width="60%" height="30%">
 
📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
⬅️ [Previous notebook](./ex_07_ray_data.ipynb) <br>
 
### Learning objectives:
  * Get introduced to Ray AIR as a unified toolkit to write an end-to-end ML application in a single Python script
  * Get exposed to Ray data for data ingestion
  * Use out-of-box Preprocessors
  * Load model from the best model checkpoint and use for batch inference
  * Deploy best checkpoint model and use for online inference

In [14]:
import logging, os, random, warnings
import ray
import pandas as pd

In [15]:
warnings.filterwarnings("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"

In [16]:
if ray.is_initialized:
    ray.shutdown()
ray.init(logging_level=logging.ERROR)

Python version:,3.8.13
Ray version:,2.0.0rc0
Dashboard:,http://127.0.0.1:8266


### Create Ray data from an S3 CSV datasource

In [17]:
dataset = ray.data.read_csv("s3://anonymous@air-example-data/breast_cancer.csv")

# Split data into train and validation.
train_dataset, valid_dataset = dataset.train_test_split(test_size=0.3)
test_dataset = valid_dataset.drop_columns(["target"])

Map_Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 18.94it/s]


### Create Preprocessors
This preprocessor is automatically used in the training function to `fit` and `tranform` your datasets for training and validation. You don't
have to explicitly call the preprocess before training or inference. Ray AIR toolkit automatically does that for you. 

We are going to scaler a few features like `mean radius` and `mean texture`.

In [18]:
from ray.data.preprocessors import StandardScaler

# Create a preprocessor to scale some columns
columns_to_scale = ["mean radius", "mean texture"]
preprocessor = StandardScaler(columns=columns_to_scale)

### Create Trainers
Use the Ray AIR trainer `XGBoostTrainer` with simple steps:
 1. define the parallelism for Ray compute
 2. define the XGBoost parameters for training
 3. supply the preprocessor for fitting and transforming dataset during training and validation
 4. provide the datasets for training and validation
 5. invoke `trainer.fit()` 
 
 Simple API that does a lot behind the scenes for you!

In [19]:
from ray.air.config import ScalingConfig
from ray.train.xgboost import XGBoostTrainer

trainer = XGBoostTrainer(
    scaling_config=ScalingConfig(
        # Number of workers to use for data parallelism.
        num_workers=2,
        # Whether to use GPU acceleration.
        use_gpu=False),
    label_column="target",
    num_boost_round=20,
    params={
        # XGBoost specific params
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
    },
    # our train and validation dataset and preprocessor
    datasets={"train": train_dataset, "valid": valid_dataset},
    preprocessor=preprocessor,
)
result = trainer.fit()
# print(result.metrics)

Trial name,status,loc,iter,total time (s),train-logloss,train-error,valid-logloss
XGBoostTrainer_8ca3b_00000,TERMINATED,127.0.0.1:99676,21,4.72426,0.0184957,0,0.0893879


(XGBoostTrainer pid=99676) 2022-08-05 14:57:09,900	INFO main.py:980 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
(_RemoteRayXGBoostActor pid=99692) 2022-08-05 14:57:11,069	WARNING __init__.py:192 -- DeprecationWarning: `ray.worker.get_resource_ids` is a private attribute and access will be removed in a future Ray version.
(_RemoteRayXGBoostActor pid=99692)   File "/opt/miniconda3/envs/ray-summit-training/lib/python3.8/site-packages/ray/_private/workers/default_worker.py", line 237, in <module>
(_RemoteRayXGBoostActor pid=99692)     ray._private.worker.global_worker.main_loop()
(_RemoteRayXGBoostActor pid=99692)   File "/opt/miniconda3/envs/ray-summit-training/lib/python3.8/site-packages/ray/_private/worker.py", line 754, in main_loop
(_RemoteRayXGBoostActor pid=99692)     self.core_worker.run_task_loop()
(_RemoteRayXGBoostActor pid=99692)   File "/opt/miniconda3/envs/ray-summit-training/lib/python3.8/site-packages/ray/_private/func

Result for XGBoostTrainer_8ca3b_00000:
  date: 2022-08-05_14-57-13
  done: false
  experiment_id: be6a859dfb9741a1ad5cb8cec303898a
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 99676
  time_since_restore: 4.258946895599365
  time_this_iter_s: 4.258946895599365
  time_total_s: 4.258946895599365
  timestamp: 1659736633
  timesteps_since_restore: 0
  train-error: 0.02261306532663317
  train-logloss: 0.464117960489575
  training_iteration: 1
  trial_id: 8ca3b_00000
  valid-error: 0.11695906432748537
  valid-logloss: 0.5025240946234318
  warmup_time: 0.002750873565673828
  


(XGBoostTrainer pid=99676) 2022-08-05 14:57:13,339	INFO main.py:1516 -- [RayXGBoost] Finished XGBoost training on training data with total N=398 in 3.45 seconds (1.89 pure XGBoost training time).


Result for XGBoostTrainer_8ca3b_00000:
  date: 2022-08-05_14-57-13
  done: true
  experiment_id: be6a859dfb9741a1ad5cb8cec303898a
  experiment_tag: '0'
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 21
  node_ip: 127.0.0.1
  pid: 99676
  time_since_restore: 4.724261045455933
  time_this_iter_s: 0.3682382106781006
  time_total_s: 4.724261045455933
  timestamp: 1659736633
  timesteps_since_restore: 0
  train-error: 0.0
  train-logloss: 0.01849572773292735
  training_iteration: 21
  trial_id: 8ca3b_00000
  valid-error: 0.04093567251461988
  valid-logloss: 0.08938791319913073
  warmup_time: 0.002750873565673828
  


### Create Tuner for hyperparameter search

What if you want to do hyperparameter optimization during training and use the best config for the model?
Well, you can then use Tuner and supply your training function, Trainer, as part of the argument, along 
with other Tuner configuration. 

Again, simple steps:
 1. define your hyperparameter space
 2. define `TuneConfig` for number of trials and parallelism 
 3. invoke `tuner.fit()`

In [ ]:
from ray import tune

param_space = {"params": {"max_depth": tune.randint(1, 9)}}
metric = "train-logloss"

In [ ]:
from ray.tune.tuner import Tuner, TuneConfig
from ray.air.config import RunConfig

tuner = Tuner(
    trainer,
    param_space=param_space,
    tune_config=TuneConfig(num_samples=5, metric=metric, mode="min"),
)
# Execute tuning.
result_grid = tuner.fit()

In [ ]:
# Fetch the best result with its best hyperparameter config 
best_result = result_grid.get_best_result()
print("Best Result:", best_result)

### Ray AIR Checkpoints

The AIR trainers, tuners, and custom pretrained model generate Checkpoints. An AIR Checkpoint is a common format for models that are used across different components of the Ray AI Runtime. This common format allow easy interoperability among AIR components and seamless integration with external supported machine learning frameworks. Read more
about [Checkpoints]().

<img src="images/checkpoints.jpeg" height="25%" and width="50%"> 

### Create a `BatchPreditor` for batch prediction
Once you have trained and tuned your model, create a batch predictor from best model using the `best_result.checkpoint` and do batch inference. 

In [ ]:
from ray.train.batch_predictor import BatchPredictor
from ray.train.xgboost import XGBoostPredictor

batch_predictor = BatchPredictor.from_checkpoint(best_result.checkpoint, XGBoostPredictor)

predicted_probabilities = batch_predictor.predict(test_dataset)
print("PREDICTED PROBABILITIES")
predicted_probabilities.show()

### Create `PredictorDeployment` for Online Inference

Deploy the best model as an inference service by using Ray Serve and the `PredictorDeployment` class.

In [ ]:
from ray import serve
from fastapi import Request
from ray.serve import PredictorDeployment
from ray.serve.http_adapters import json_request


async def adapter(request: Request):
    content = await request.json()
    print(content)
    return pd.DataFrame.from_dict(content)


serve.start(detached=True)
deployment = PredictorDeployment.options(name="XGBoostService", num_replicas=2, route_prefix="/rayair")

deployment.deploy(
    XGBoostPredictor, best_result.checkpoint, batching_params=False, http_adapter=adapter
)

print(deployment.url)

After deploying the service, you can send requests to it.

In [ ]:
import requests

sample_input = test_dataset.take(1)
sample_input = dict(sample_input[0])

output = requests.post(deployment.url, json=[sample_input]).json()
print(output)

In [ ]:
ray.shutdown()

### Homework

1. Have a go at Ray AIR examples in the documentation.

 📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
⬅️ [Previous notebook](./ex_07_ray_data.ipynb) <br>

Done! 🍻
 